In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [3]:
df = pd.read_csv('data.csv')

df.columns = df.columns.str.lower().str.replace(' ','_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)
for col in categorical_columns:
    df[col] = df[col].str.lower().str.replace(' ','_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [4]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [5]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']
categorical =['gender',
              'seniorcitizen',
              'partner',
              'dependents', 
              'phoneservice',
              'multiplelines',
              'internetservice', 
              'onlinesecurity',
              'onlinebackup', 
              'deviceprotection', 
              'techsupport', 
              'streamingtv', 
              'streamingmovies', 
              'contract', 
              'paperlessbilling', 
              'paymentmethod']

In [6]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient = 'records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=5000, solver='lbfgs')
    model.fit(X_train, y_train)

    return dv, model

In [7]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient = 'records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:,1]

    return y_pred

In [8]:
C = 1.0
n_splits = 5

In [9]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
        
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]
    
    y_train = df_train.churn.values
    y_val = df_val.churn.values
    
    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)
    
    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

    print('%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

1.0 0.844 +- 0.000
1.0 0.845 +- 0.000
1.0 0.841 +- 0.005
1.0 0.839 +- 0.005
1.0 0.842 +- 0.007


In [10]:
scores

[0.8443375120691413,
 0.8449977651966627,
 0.8333012241621514,
 0.8347768913474451,
 0.8518166985660947]

In [11]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1)
y_pred = predict(df_test, dv, model)

y_test = df_test.churn.values
auc = roc_auc_score(y_test, y_pred)
auc

0.8584032088573997

<h3>Saving the model</h3>

In [12]:
import pickle 

In [13]:
output_file = f'model_c={C}.bin'
output_file

'model_c=1.0.bin'

In [14]:
f_out = open(output_file, 'wb')
pickle.dump((dv, model), f_out)
f_out.close()

In [15]:
# This with help you if you forget to close the file you could work with the file only under the 'with'
with open(output_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)

<h3>Loading the model</h3>

In [16]:
import pickle

In [17]:
model_file = 'model_c=1.0.bin'

In [18]:
# This with help you if you forget to close the file you could work with the file only under the 'with'
with open(model_file, 'rb') as f_in:
    (dv, model) = pickle.load(f_in)

In [19]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(C=1, max_iter=5000))

In [20]:
customer = {
        "customerid": "5442-pptjy",
        "gender": "male",
        "seniorcitizen": 0,
        "partner": "yes",
        "dependents": "yes",
        "tenure": 12,
        "phoneservice": "yes",
        "multiplelines": "no",
        "internetservice": "no",
        "onlinesecurity": "no_internet_service",
        "deviceprotection": "no_internet_service",
        "techsupport": "no_internet_service",
        "streamingtv": "no_internet_service",
        "streamingmovies": "no_internet_service",
        "contract": "two_year",
        "paperlessbilling": "no",
        "paymentmethod": "mailed_check",
        "monthlycharges": 19.7,
        "totalcharges": 258.35,
}

In [21]:
X = dv.transform([customer])

In [22]:
model.predict_proba(X)[0,1]

np.float64(0.03614647849853299)